# NLP Challenge

## 1. Classifier

>You just trained a model to classify soft drinks according to their flavor-profile. The results are below.
>- What has the model learned about the correspondence between the item name and the classification?
>- Can you find any problems with what the model has learned?
>- How would you address this problem?

In [1]:
import pandas as pd

In [2]:
results_df = pd.DataFrame(data={
    "item_name": [
        "7up can 355ml", "pepsi bottle 14oz", "7up bottle 500ml", "7up can 500ml",
        "pepsi bottle 20oz", "coke bottle 14oz", "pepsi can 20oz", "coke bottle 20oz",
        "sprite bottle 500ml", "sprite can 500ml", "sprite can 355ml", "coke can 20oz",
        "sprite bottle 355ml", "7up bottle 355ml", "pepsi can 14oz", "coke can 14oz"
        ],
    "flavor_profile": [
        "lemon-line", "cola", "lemon-line", "lemon-line", "cola", "cola", "cola",
        "cola", "lemon-line", "lemon-line", "lemon-line", "cola", "lemon-line",
        "lemon-line", "cola", "cola"
        ]})

results_df

,item_name,flavor_profile
0,7up can 355ml,lemon-line
1,pepsi bottle 14oz,cola
2,7up bottle 500ml,lemon-line
3,7up can 500ml,lemon-line
4,pepsi bottle 20oz,cola
5,coke bottle 14oz,cola
6,pepsi can 20oz,cola
7,coke bottle 20oz,cola
8,sprite bottle 500ml,lemon-line
9,sprite can 500ml,lemon-line


> What has the model learned about the correspondence between the item name and the classification?

It appears as though the model has learned to classify on the brand name alone. All 7up and Sprite are lemon-line while all pepsi and coke are cola regardless of the container type and size.

## 2. Brand Extraction

> You’ve been given a product list of ~16,000 carbonated soft drinks (see carbonated_soft_drinks.csv) and you’re asked to build a model that can extract the product’s brand (e.g., Coke, Pepsi Sunkist), when available.
>- What type of model would you use?
>- If you choose a supervised-learning model, where will you get your training data?

In [5]:
soft_drinks_df = pd.read_csv('../data/carbonated_soft_drinks.csv')

In [6]:
soft_drinks_df

,item_name
0,Bottle Coke Classic 20oz
1,Bottle Coke Diet 20oz
2,20oz Fountain Beverage
3,Bottle Pepsi 20oz
4,32oz Fountain Beverage
...,...
16979,Large Dr. Pepper BOOST
16980,Large Coca-Cola BOOST
16981,Diet Ginger Ale
16982,UpSZ Soda


This is an interesting problem. The question seems to indicate that ML is necessary, however, I cannot recall using any model to extract from text.

Models make predictions. Predict which word in the product name is the brand. If the model is over a confidence threshold (say 50%) copy that name to another list.


My initial response to this problem is not to bother with ML.

- Use RegEx to extract components like volume
- Use a client list to identify all brand names that may exist in the list
  - fuzzy lookup to see if the brand exists in the product name and account for typos

However, I can see a problem with that approach since it would be near impossible to create an exhaustive regex expression to account for all possible variations and mistakes.